In [1]:
#from __future__ import division, print_function, absolute_import

import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [4]:
# Training Parameters
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [5]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')




In [6]:
# Store layers weight & bias
weights = {
    # 3x3 conv, 1 input, 16 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 16])),  #filter_height, filter_width, in_channels, out_channels
    # 3x3 conv, 16 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([3, 3, 16, 64])), 
    # fully connected, 7*7*64 inputs, 512 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 512])),
    # 512 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([512, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([16])),
    'bc2': tf.Variable(tf.random_normal([64])), 
    'bd1': tf.Variable(tf.random_normal([512])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [7]:
# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1]) # -1 refers to standard feature which is equivalent to 28*28*1 here

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1']) #28x28x16
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2) #14x14x16

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2']) #14x14x64
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)#7x7x64

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1']) #512 nodes
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout) #dropped out

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out']) #10 nodes
    return out  #10 nodes

In [8]:
# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = optimizer.minimize(loss)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [9]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 0.8})
            print("step " + str(step) + ",Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))

step 1,Loss= 18024.7754, Training Accuracy= 0.102
step 10,Loss= 8412.4922, Training Accuracy= 0.195
step 20,Loss= 6735.8457, Training Accuracy= 0.250
step 30,Loss= 6426.6802, Training Accuracy= 0.352
step 40,Loss= 4448.5503, Training Accuracy= 0.359
step 50,Loss= 2546.4536, Training Accuracy= 0.586
step 60,Loss= 2453.9492, Training Accuracy= 0.547
step 70,Loss= 2170.6470, Training Accuracy= 0.578
step 80,Loss= 2166.0894, Training Accuracy= 0.609
step 90,Loss= 1943.6326, Training Accuracy= 0.656
step 100,Loss= 1945.5300, Training Accuracy= 0.594
step 110,Loss= 1174.6321, Training Accuracy= 0.641
step 120,Loss= 1269.5830, Training Accuracy= 0.742
step 130,Loss= 1881.0305, Training Accuracy= 0.711
step 140,Loss= 899.5845, Training Accuracy= 0.773
step 150,Loss= 956.1780, Training Accuracy= 0.766
step 160,Loss= 805.5683, Training Accuracy= 0.789
step 170,Loss= 968.2585, Training Accuracy= 0.766
step 180,Loss= 610.0254, Training Accuracy= 0.750
step 190,Loss= 900.8807, Training Accuracy= 0.